In [23]:

import speech_recognition as sr
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.utilities import OpenWeatherMapAPIWrapper,WikipediaAPIWrapper
from langchain.agents import Tool
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import create_react_agent, AgentExecutor


In [3]:
from dotenv import load_dotenv
_ = load_dotenv()
recognizer = sr.Recognizer()

In [4]:
weather_api = OpenWeatherMapAPIWrapper()  #https://home.openweathermap.org/api_keys

wiki_api = WikipediaAPIWrapper()


In [5]:
embed = OpenAIEmbeddings(model='text-embedding-ada-002') 

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    

In [6]:
def wiki_search(query):
    data = wiki_api.load(query)
    split_docs = splitter.split_documents(data)
    vectorstore = FAISS.from_documents(split_docs, embed)

    return vectorstore.as_retriever()

In [7]:
def retriever_tool(query):
    retriever = wiki_search(query)
    results = retriever.get_relevant_documents(query)
    return "\n".join([doc.page_content for doc in results])


In [8]:
from langchain.llms import OpenAI
# from google.colab import userdata
# api_key = userdata.get('OPENAI_API')
llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo-instruct")  # or "gpt-4"


C:\Users\riina\AppData\Local\Temp\ipykernel_21676\89507056.py:4: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo-instruct")  # or "gpt-4"


In [9]:
tools = [
    Tool(
    name="TravelInfoRetriever",
    func=retriever_tool,
    description=(
        "Use this tool to retrieve travel-related information about countries, cities, or landmarks. "
        "Ideal when the user asks something like: 'What is interesting in Paris?', or 'What is Tokyo known for?'."
    )
    ),

  Tool(
        name='Weather',
        func= weather_api.run,
        description=(
        "Use this tool to find **current or forecasted weather information** about a country, city, or travel destination. "
        "Ideal for questions like: 'What's the weather in Rome?', 'Is it rainy in Tokyo?', or 'How cold is it in Iceland in December?'. "
        "Only use this tool when the user is asking specifically about **weather** conditions. "
        "Do not use it for general travel info or sightseeing.")
  )
]

In [10]:
def voice_to_text():
    try:
        with sr.Microphone() as source:
            print("\nListening... (Speak now)")
            audio = recognizer.listen(source, timeout=10)
        return recognizer.recognize_google(audio)
    except sr.WaitTimeoutError:
        print("Listening timed out")
        return ""
    except Exception as e:
        print(f"Audio error: {str(e)}")
        return ""

def get_user_input():
    input_type = input("\nChoose input method [text/audio]: ").lower()
    return voice_to_text() if input_type == "audio" else input("Your question: ")


In [17]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
)

In [13]:
zero_shot_agent.run(get_user_input())


Listening... (Speak now)


C:\Users\riina\AppData\Local\Temp\ipykernel_30936\2041125750.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  zero_shot_agent.run(get_user_input())




> Entering new AgentExecutor chain...
 I should use the TravelInfoRetriever tool to find out what London is.
Action: TravelInfoRetriever
Action Input: London

C:\Users\riina\AppData\Local\Temp\ipykernel_30936\741416126.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)



Observation: Paul's Cathedral, Tower Bridge, and Trafalgar Square. The city has the most museums, art galleries, libraries, and cultural venues in the UK, including the British Museum, National Gallery, Natural History Museum, Tate Modern, British Library, and numerous West End theatres. Important sporting events held in London include the FA Cup Final, the Wimbledon Tennis Championships, and the London Marathon. It became the first city to host three Summer Olympic Games upon hosting the 2012 Summer
As one of the world's major global cities, London exerts a strong influence on world art, entertainment, fashion, commerce, finance, education, healthcare, media, science, technology, tourism, transport, and communications. London is Europe’s most economically powerful city, and is one of the world's major financial centres. London hosts Europe's largest concentration of higher education institutions, comprising over 50 universities and colleges and enrolling more than 500,000 students as

'London is a major global city with a rich history, diverse culture, and strong influence on various industries. It is known for its iconic landmarks, museums, and universities. The current weather in London is scattered clouds with a temperature of 10.59°C.'